# Chapter 1: Introduction

## 1.1 Overview
This project aims to perform flood detection using Sentinel-1 SAR imagery and Google Earth Engine (GEE). The area of interest (AOI) is Cork, Ireland, which experienced significant flooding in 2023.

## 1.2 Objectives
- Set up the environment and initialize Google Earth Engine.
- Define the Area of Interest (AOI) and fetch satellite images.
- Process and visualize the data to detect changes in water-covered areas.
- Export the processed images for further analysis.


# Chapter 2: Setting Up the Environment

## 2.1 Importing Libraries & Starting Earth Engine
In this phase, we are setting up Google Earth Engine to fetch and process satellite images for a specific area and time range.

## 2.2 Defining the Area of Interest (AOI)
According to The Irish Times: https://www.irishtimes.com/ireland/2023/10/19/cork-flooding-floods-in-co-cork-absolutely-devastating-as-safety-warning-issued-to-motorists/
Picked the area that got more affected with the floods of 2023.3

This phase is crucial for gathering the necessary data to analyze changes in water-covered areas, which is essential for flood detection.


In [1]:
import ee
import geemap

# Initialize GEE
ee.Authenticate()
ee.Initialize(project='atu-fyp')

# Chapter 3: Data Collection and Processing

## 3.1 Setting Up Google Earth Engine
Defining the Area of Interest (AOI): Specifying the geographic region.
Fetching Satellite Images: Retrieving Landsat and Sentinel-2 images for the defined AOI and time range, with filters for cloud cover.
Visualizing the Data: Adding the median composite images to an interactive map for visualization.
Exporting the Data: Exporting the processed images to Google Drive for further analysis.
This phase is crucial for gathering the necessary data to analyze changes in water-covered areas, which is essential for flood detection.

## 3.2 Setting Area Chosen
Setting up the environment.

In [2]:
# Define AOI for Cork
# Define pre-flood and post-flood dates
#We tell the computer when to look.
#We check images before the flood (September) and after the flood (October).
aoi = ee.Geometry.Polygon([
    [-8.570156845611686, 52.00904254772663],
    [-8.570156845611686, 51.7292195887807],
    [-7.959042343658562, 51.7292195887807],
    [-7.959042343658562, 52.00904254772663]
]).buffer(5000)  # Buffer to expand the area

# Initialize the map
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)
Map.addLayer(aoi, {'color': 'red'}, 'Area of Interest')
Map

Map(center=[51.8691, -8.2646], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

# Chapter 4: Loading and Visualizing Data

## 4.1 Loading Sentinel-1 Data
Load Sentinel-1 GRD data for pre-flood and post-flood periods.

In [3]:
# Defines pre-flood and post-flood dates
pre_flood_start = '2023-09-01'
pre_flood_end = '2023-09-30'
post_flood_start = '2023-10-18'
post_flood_end = '2023-10-25'
# Load Sentinel-1 GRD data for pre-flood period
pre_flood_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(pre_flood_start, pre_flood_end) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select('VV')

# Load Sentinel-1 GRD data for post-flood period
post_flood_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select('VV')

# Get the median image for each period
pre_flood_image = pre_flood_collection.median()
post_flood_image = post_flood_collection.median()

# Add pre-flood and post-flood images to the map
Map.addLayer(pre_flood_image.clip(aoi), {'min': -20, 'max': 0}, 'Pre-Flood VV')
Map.addLayer(post_flood_image.clip(aoi), {'min': -20, 'max': 0}, 'Post-Flood VV')
Map.centerObject(aoi)
Map

Map(center=[51.8691, -8.2646], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

# Chapter 5: Analyzing Changes

## 5.1 Calculate the Difference Between Pre-Flood and Post-Flood Images
Subtracting the pre-flood image from the post-flood image to detect changes.

This cell calculates the difference between the post-flood and pre-flood images by subtracting the pre-flood image from the post-flood image. The resulting difference image highlights changes in the area. The difference image is then added to the map with a color palette to visualize the changes.

In [4]:
# Calculate the difference between post-flood and pre-flood images
difference_image = post_flood_image.subtract(pre_flood_image)

# Add the difference image to the map
Map.addLayer(difference_image.clip(aoi), {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}, 'Difference Image')
Map.centerObject(aoi)
Map


Map(center=[51.86932901232889, -8.264599594635124], controls=(WidgetControl(options=['position', 'transparent_…

## 5.2 Thresholding to Identify Flooded Areas
Applying a threshold to the difference image to identify flooded areas.
This code defines a threshold value for flood detection. The gt(threshold) function creates a binary flood mask by identifying pixels in the difference image that exceed the threshold value. The flood mask is then added to the map with a color palette to visualize the flooded areas.

In [5]:
# Define a threshold for flood detection
threshold = 2  # Adjust this value based on your analysis

# Create a binary flood mask
flood_mask = difference_image.gt(threshold)

# Add the flood mask to the map
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'Flood Mask')
Map.centerObject(aoi)
Map

Map(center=[51.86932901232889, -8.264599594635124], controls=(WidgetControl(options=['position', 'transparent_…

# Chapter 6: Exporting and Visualizing Results

## 6.1 Export the Flood Mask
Exporting the flood mask to Google Drive for further analysis.



In [6]:
# Export the flood mask to Google Drive
task = ee.batch.Export.image.toDrive(
    image=flood_mask.clip(aoi),
    description='FloodMask',
    folder='EarthEngineImages',
    scale=30,
    region=aoi
)
task.start()

print("Export task started. Check Google Drive for the flood mask.")

Export task started. Check Google Drive for the flood mask.


## 6.2 Visualizing Locally
Visualizing the exported flood mask locally.

Visualizing locally

In [7]:
import rasterio
from matplotlib import pyplot as plt

# Check if the file exists
import os
file_path = r"C:\\Users\\Neo\\Desktop\\FYP\\geeimages\\FloodMask.tif"
if os.path.exists(file_path):
    # Open the exported flood mask
    with rasterio.open(file_path) as src:
        flood_mask_image = src.read(1)

    # Display the flood mask
    plt.imshow(flood_mask_image, cmap='Blues', vmin=0, vmax=1)
    plt.colorbar(label='Flood Mask')
    plt.title('Flood Mask')
    plt.show()
else:
    print(f"File not found: {file_path}")

File not found: C:\\Users\\Neo\\Desktop\\FYP\\geeimages\\FloodMask.tif


# Chapter 7: Refining the Threshold
## 7.1 Experiment with different threshold values (e.g., 1.5, 2.5) to improve flood detection.

In [8]:
# Defining a list of threshold values to test
thresholds = [1.5, 2, 2.5, 3]

# map to visualize the results
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)

# Loops through each threshold and add the flood mask to the map
for threshold in thresholds:
    flood_mask = difference_image.gt(threshold)
    Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, f'Flood Mask (Threshold={threshold})')

# Adds the AOI and difference image for reference
Map.addLayer(aoi, {'color': 'red'}, 'Area of Interest')
Map.addLayer(difference_image.clip(aoi), {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}, 'Difference Image')
Map.centerObject(aoi)
Map

Map(center=[51.8691, -8.2646], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

EXPLANATION

Imagine you have a list of numbers that you want to test. These numbers are called "thresholds". In this case, the thresholds are [1.5, 2, 2.5, 3].

Creating a Map:
Think of a map like a big piece of paper where you can draw and see different things. Here, we are creating a map centered on a place called Cork, Ireland.

Looping Through Thresholds:
We are going to look at each number in our list of thresholds one by one.
For each number (threshold), we are going to create something called a "flood mask". This is like putting a special filter on the map to see where the water is.
We then add this flood mask to our map. It's like drawing on the map with different colors to show where the water is based on each threshold.

# Chapter 8: Combining Sentinel-1 and Sentinel-2 DATA
## 8.1 Sentinel-2 optical data used to calculate the Normalized Difference Water Index (NDWI), which is useful for validating the flood mask derived from Sentinel-1.

In [9]:

# Loading Sentinel-2 data for the post-flood period
#We ask Google Earth Engine for satellite images from Sentinel-2, a satellite that takes pictures of Earth. 
#We only take images from our area (Cork) and only in October.
#We ignore pictures that have too many clouds (less than 50% clouds allowed). 
post_flood_sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))  # Filter out cloudy images

# Function to calculate NDWI
#NDWI (Normalized Difference Water Index) is a math trick to find water in a satellite image. 
# It looks at two colors:
# B3 (Green light) 
# B8 (Near Infrared light) 
# If B3 is bright and B8 is dark, that means water! 
# We add this water-detection layer to the image.
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Apply NDWI calculation to the collection
ndwi_collection = post_flood_sentinel2.map(add_ndwi)

# Gets the median NDWI image
ndwi_image = ndwi_collection.median()

# Adds NDWI layer to the map
Map.addLayer(ndwi_image.clip(aoi), {'bands': ['NDWI'], 'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')
ndwi_vis_params = {'bands': ['NDWI'], 'min': -0.3, 'max': 0.3, 'palette': ['white', 'blue']}
Map.addLayer(ndwi_image.clip(aoi), ndwi_vis_params, 'NDWI')

Map.centerObject(aoi)
Map

Map(center=[51.86932901232889, -8.264599594635124], controls=(WidgetControl(options=['position', 'transparent_…

# Chapter 9:  VALIDATING the FLOOD MASK
## 9.1 Comparing the flood mask derived from Sentinel-1 with the NDWI from Sentinel-2 to validate the results.

In [10]:
# Defines a threshold for NDWI (values > 0.3 indicate water)
ndwi_threshold = 0.3
ndwi_water_mask = ndwi_image.select('NDWI').gt(ndwi_threshold)

# Adds the NDWI water mask to the map
Map.addLayer(ndwi_water_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI Water Mask')

# Compares with the Sentinel-1 flood mask
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Sentinel-1 Flood Mask')
Map.centerObject(aoi)
Map

Map(center=[51.86932901232889, -8.264599594635124], controls=(WidgetControl(options=['position', 'transparent_…

EXPLANATION

This code creates a mask (a special filter) to highlight only the water areas on the map. It looks at the NDWI values and decides which pixels are water and which are not water.

ndwi_threshold = 0.3
ndwi_water_mask = ndwi_image.select('NDWI').gt(ndwi_threshold)

What this does:
We set a "rule" : Any place with an NDWI value higher than 0.3 is water.
ndwi_image.select('NDWI') takes the NDWI values.
.gt(ndwi_threshold) means "greater than 0.3".
The result is a black & white mask:
White (1) = Water 
Black (0) = Not Water 

ANALOGY
Imagine you have a coloring book. You look at each spot on the page and decide:

If it's NDWI > 0.3, you color it blue (water)
If it's NDWI ≤ 0.3, you leave it white (land)



Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Sentinel-1 Flood Mask')

Sentinel-1 is another satellite (different from Sentinel-2).
This flood mask is another way to detect water.
Green = Flooded areas detected by Sentinel-1. 

 Analogy:
Imagine you have two different cameras  looking at the same flood:

NDWI from Sentinel-2 (Blue)
Flood detection from Sentinel-1 (Green)
This helps you compare the two methods to see if they agree!

# Chapter 10: VALIDATION RESULTS

In [11]:
# Exports NDWI water mask
task_ndwi = ee.batch.Export.image.toDrive(
    image=ndwi_water_mask.clip(aoi),
    description='NDWI_WaterMask',
    folder='EarthEngineImages',
    scale=10,
    region=aoi
)
task_ndwi.start()

# # Exports Sentinel-1 flood mask
# task_flood = ee.batch.Export.image.toDrive(
#     image=flood_mask.clip(aoi),
#     description='Sentinel1_FloodMask',
#     folder='EarthEngineImages',
#     scale=30,
#     region=aoi
# )
# task_flood.start()

print("Export tasks started. Check Google Drive for the validation results.")

Export tasks started. Check Google Drive for the validation results.


In [12]:
# Load Sentinel-2 imagery for the post-flood period
post_flood_sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 0))  # Filter out cloudy images

# Get the first Sentinel-2 image for visualization
sentinel2_image = post_flood_sentinel2.first()

# Add Sentinel-2 true color image to the map
Map.addLayer(sentinel2_image.clip(aoi), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 True Color')

# Add the flood mask and NDWI water mask for comparison
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Sentinel-1 Flood Mask')
Map.addLayer(ndwi_water_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI Water Mask')

# Add the AOI for reference
Map.addLayer(aoi, {'color': 'red'}, 'Area of Interest')
Map.centerObject(aoi)
Map

Map(center=[51.86932901232889, -8.264599594635124], controls=(WidgetControl(options=['position', 'transparent_…

No such comm: 30a8b1743853406dab1395598506f713
No such comm: 565464a312004d12b2dc16b6093a25a0
No such comm: 30a8b1743853406dab1395598506f713
No such comm: 30a8b1743853406dab1395598506f713
No such comm: 30a8b1743853406dab1395598506f713


In [13]:
# Load ground truth flood map (replace with your ground truth data)
# For demonstration, we'll use the NDWI water mask as ground truth
ground_truth = ndwi_water_mask

# Calculate confusion matrix
confusion_matrix = flood_mask.add(ground_truth.multiply(2)).clip(aoi)

# Add confusion matrix to the map
Map.addLayer(confusion_matrix, {'min': 0, 'max': 3, 'palette': ['white', 'green', 'blue', 'red']}, 'Confusion Matrix')
Map.centerObject(aoi)
Map

Map(center=[51.86932901232889, -8.264599594635124], controls=(WidgetControl(options=['position', 'transparent_…

# Chapter: Comparing with Ground Truth

In [15]:
#Layer NDWI AND SENTINEL-1 FLOOD MASK

# Loads Sentinel-2 data for the post-flood period
post_flood_sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Filters out cloudy images

# Function to calculate NDWI
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Applies NDWI calculation to the collection
ndwi_collection = post_flood_sentinel2.map(add_ndwi)

# Gets the median NDWI image
ndwi_image = ndwi_collection.median()

# Defines a threshold for NDWI (values > 0.3 indicate water)
ndwi_threshold = 0.3
ndwi_water_mask = ndwi_image.select('NDWI').gt(ndwi_threshold)

# Adds NDWI water mask to the map
Map.addLayer(ndwi_water_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI Water Mask')

# Compares with Sentinel-1 flood mask
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Sentinel-1 Flood Mask')
Map.centerObject(aoi)
Map

Map(bottom=87102.0, center=[51.82050047836314, -8.298797607421877], controls=(WidgetControl(options=['position…

EXPLANATION

The NDWI water mask (blue) represents water bodies detected by Sentinel-2.

The Sentinel-1 flood mask (green) represents flooded areas detected by Sentinel-1.

Overlaying the two masks to see how well they align.

Here we look for areas where both masks (blue and green) overlap. These are likely true flooded areas.

Areas where only one mask detects water may indicate false positives or false negatives.

Visual Comparison
